# Tissue Classification using Neural Networks
In this lab we will explore the use of texture in images and traditional machine learning approaches such as clustering. The dataset we will be using is available here: http://dx.doi.org/10.5281/zenodo.53169. 

![alt text](https://www.researchgate.net/profile/Jakob_Kather/publication/303998214/figure/fig7/AS:391073710002224@1470250646407/Representative-images-from-our-dataset-Here-the-first-10-images-of-every-tissue-class.png)

The above figure shows the 8 different classes of tissue we will be trying to identify. 

In [0]:
# Imports
from __future__ import print_function
import os
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

## Step 1
* Load the data (done for you)
 * The "data" variable stores 5000 images of shape 150x150. This means data has shape (5000, 150, 150). These images are loaded here as grayscale.
 * The "labels" variable stores 5000 labels (0-7). This means "labels" has shape (5000,)
* Split data into training and testing subsets (left up to you)
 * Check out the sklearn function train_test_split from sklearn.model_selection

In [0]:
#! git clone https://github.com/BeaverWorksMedlytics/Week3_public.git

# Build the path to the data folder. No need to change directories
# There are a total of 6 files you will have to load
data_dir = os.path.join( os.getcwd(), 'Week3_public', 'data', 'crc')

In [4]:
# Load data and split into training, testing sets
y = np.load(os.path.join(data_dir, 'rgb01.npz'))
labels = y['labels']
data = y['rgb_data']
data = data[:,0:64,0:64,0]
label_str = y['label_str']
label_str = label_str.tolist() # this is to convert label_str back to a dictionary
y = []

print(data.shape)
for ii in range(2,6):
    filename = os.path.join(data_dir, 'rgb0' + str(ii) + '.npz')
    print('loading ', filename)
    y = np.load(filename)
    labels = np.append(labels, y['labels'], axis=0)
    data = np.append(data, y['rgb_data'][:,0:64,0:64,0], axis=0)
    print(data.shape)
    y = []


print( data.shape )
print( labels.shape )

FileNotFoundError: ignored

In [0]:
num_images, nrows, ncols = data.shape

# split into training and testing sets
testsize = .2
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size= testsize)

# convert the labels from 1-D arrays to categorical type 
train_data = train_data.reshape(int((1-testsize)*num_images), nrows*ncols)
test_data = test_data.reshape(int((testsize)*num_images), nrows*ncols)


## Normalize data
All images should be normalized to the range 0-1 by dividing by 255.

#### Note
* Using the La\*b colorspace : If you convert your images to the La\*b colorspace, the scaling factor will change. Each channel in this colorspace will have a different range and normalization of each space will involve scaling each channel separately. Additionally, the a\* channel can have a negative range. This also needs to be taken into account. 
* Using the HSV/HSI colorspace : Similar considerations apply if you are using the HSV/HSI colorspace. The only difference is that the HSV/HSI colorspace will have all positive values.

In [0]:
# Assuming we are using the RGB colorspace
# Normalize all images so that they are 0-1
train_data = train_data.astype("float")/255
test_data = test_data.astype("float")/255


train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## Step 2
At this point, the data has been split into training and testing sets and normalized. We will now design a fully connected neural network for texture classification. 


![alt text](http://adventuresinmachinelearning.com/wp-content/uploads/2017/04/CNN-example-block-diagram.jpg)


( Image from http://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/ )

When designing a fully connected network for classification, we have several decisions to make.

**Network Architecuture**
* How many layers will our network have ?
* How many convolutional filters per layer ?
    * What is an appropriate filter size ? 
* What is an appropriate batch size, learning rate and number of training epochs ?

**Data input**
* Do we use the raw data ?
    * RGB or just gray channel ?
* Does the use of different colorspaces lead to better results for a given network architecture ?
* Can we use any of the texture features from the previous lab as inputs to this model ?
* How does data augmentation affect the results ? 

Other considerations, we will not be exploring :
* What is the trade-off between input data sizes and batch size ?
* Is the GPU always the appropriate platform for training ?
* How does hardware influence inputs and batch sizes for a given desired accuracy ?

In [0]:
# Define the data shapes based on your decision to use rgb or grayscale or other colorpsaces or texture features or 
# some combination of these inputs
num_classes = 8 
input_shape = nrows, ncols, 1
train_data = train_data.reshape(train_data.shape[0], nrows, ncols, 1)
test_data = test_data.reshape(test_data.shape[0], nrows, ncols, 1)

## Step 3
Design your network here using Keras

In [1]:
import tensorflow as tf
import keras

# Create your network
model = []
model = Sequential()

# Add input layer

model.add(tf.keras.layers.Convolution2D(filters=18, kernel_size=(3,3), input_shape=input_shape, activation='relu'))# add convolutional layer )
model.add(tf.keras.layers.Convolution2D(filters=18, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2))) # add pooling layer )
model.add(tf.keras.layers.Dropout(.2))
model.add(tf.keras.layers.Convolution2D(filters=18, kernel_size=(3,3), activation='tanh'))
model.add(tf.keras.layers.Convolution2D(filters=18, kernel_size=(3,3), activation='tanh'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2))) # add pooling layer )
model.add(tf.keras.layers.Dropout(.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='sigmoid')) # what layer is needed to output the predict results )
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

# Add fully connected layers 
# See keras.io for Conv2D, MaxPool2D, Dropout documentation

# Add final output layer - This should have as many neurons as the number
# of classes we are trying to identify
#model.add( )

model.summary()


Using TensorFlow backend.


NameError: ignored

## Step 4
Compile the model you designed. Compiltation of the Keras model results in the initialization of model weights and sets other model properties.

In [0]:
lr = .00001 # choose a learning rate, this will be a good paramter to tune
model.compile( optimizer=tf.train.AdamOptimizer(lr), loss=keras.losses.categorical_crossentropy, metrics = ['accuracy'] )

In [9]:
print (train_labels)

[[0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


## Step 5
Train model

In [0]:
#y = model.fit(???)
history = model.fit(train_data,
                    train_labels,
                    epochs= 100,
                    batch_size= 50,
                    validation_data=(test_data, test_labels),  # your validation data, val labels),
                    verbose=1 
                    )

Train on 4000 samples, validate on 1000 samples
Epoch 1/100
4000/4000 [==============================] - 38s 9ms/step - loss: 0.4464 - acc: 0.8505 - val_loss: 1.2013 - val_acc: 0.5500
Epoch 2/100
4000/4000 [==============================] - 38s 10ms/step - loss: 0.4310 - acc: 0.8530 - val_loss: 1.2516 - val_acc: 0.5740
Epoch 3/100
4000/4000 [==============================] - 38s 9ms/step - loss: 0.3017 - acc: 0.9105 - val_loss: 1.2840 - val_acc: 0.5750
Epoch 4/100
2900/4000 [====================>.........] - ETA: 9s - loss: 0.2715 - acc: 0.9231

4000/4000 [==============================] - 38s 9ms/step - loss: 0.2586 - acc: 0.9292 - val_loss: 1.4081 - val_acc: 0.5610
Epoch 5/100
4000/4000 [==============================] - 38s 10ms/step - loss: 0.2660 - acc: 0.9135 - val_loss: 1.4391 - val_acc: 0.5680
Epoch 6/100
4000/4000 [==============================] - 38s 10ms/step - loss: 0.1954 - acc: 0.9452 - val_loss: 1.5036 - val_acc: 0.5560
Epoch 7/100
4000/4000 [==============================] - 38s 10ms/step - loss: 0.1761 - acc: 0.9512 - val_loss: 1.5442 - val_acc: 0.5560
Epoch 8/100
 900/4000 [=====>........................] - ETA: 26s - loss: 0.1336 - acc: 0.9622

4000/4000 [==============================] - 38s 10ms/step - loss: 0.1340 - acc: 0.9660 - val_loss: 1.6125 - val_acc: 0.5640
Epoch 9/100
4000/4000 [==============================] - 38s 10ms/step - loss: 0.1344 - acc: 0.9622 - val_loss: 1.6333 - val_acc: 0.5590
Epoch 10/100
4000/4000 [==============================] - 38s 9ms/step - loss: 0.1195 - acc: 0.9673 - val_loss: 1.7110 - val_acc: 0.5550
Epoch 11/100
4000/4000 [==============================] - 38s 9ms/step - loss: 0.1286 - acc: 0.9630 - val_loss: 1.6573 - val_acc: 0.5520
Epoch 12/100
 600/4000 [===>..........................] - ETA: 29s - loss: 0.1121 - acc: 0.9733

4000/4000 [==============================] - 38s 9ms/step - loss: 0.1048 - acc: 0.9715 - val_loss: 1.7853 - val_acc: 0.5720
Epoch 13/100
4000/4000 [==============================] - 38s 10ms/step - loss: 0.0962 - acc: 0.9728 - val_loss: 1.7283 - val_acc: 0.5700
Epoch 14/100
4000/4000 [==============================] - 38s 9ms/step - loss: 0.0859 - acc: 0.9770 - val_loss: 1.8956 - val_acc: 0.5570
Epoch 15/100
4000/4000 [==============================] - 38s 10ms/step - loss: 0.1261 - acc: 0.9595 - val_loss: 1.8162 - val_acc: 0.5640
Epoch 16/100
 550/4000 [===>..........................] - ETA: 30s - loss: 0.1040 - acc: 0.9727

4000/4000 [==============================] - 38s 10ms/step - loss: 0.0923 - acc: 0.9758 - val_loss: 1.9033 - val_acc: 0.5640
Epoch 17/100
4000/4000 [==============================] - 38s 10ms/step - loss: 0.0843 - acc: 0.9748 - val_loss: 1.9502 - val_acc: 0.5400
Epoch 18/100
3400/4000 [========================>.....] - ETA: 5s - loss: 0.0878 - acc: 0.9718

## Step 6
See how your model performs by uisng it for inference.
* What is the accuracy of classification ?
* Change your model, re-compile and test. Can you improve the accuracy of the model ?


In [0]:
# predict labels - use the test set for prediction
pred_labels = model.predict(???)

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# We need to convert the categorical array test_labels into a vector
# in order to use it in the calculation of the confusion matrix
mat = confusion_matrix(np.argmax(test_labels, axis=1), pred_labels)
acc = accuracy_score(np.argmax(test_labels, axis=1), pred_labels)
print(acc)
print(mat)

In [0]:
plt.figure(figsize=(8,6))
plt.imshow(mat, cmap='hot', interpolation='nearest')
plt.grid(False)
plt.colorbar()
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

## Assignment
* In Step 3 design your own network
* Does the model perform better if you use all three RGB channels ?
* How does the performance change when using the La*b colorspace ?


In [0]:

# Load data as RGB
y = np.load(os.path.join(data_dir, 'rgb01.npz'))
labels = y['labels']
data_rgb = y['rgb_data']
label_str = y['label_str']
label_str = label_str.tolist() # this is to convert label_str back to a dictionary
y = []

print(data_rgb.shape)
for ii in range(2,6):
    filename = os.path.join(data_dir, 'rgb0' + str(ii) + '.npz')
    print('loading ', filename)
    y = np.load(filename)
    labels = np.append(labels, y['labels'], axis=0)
    data_rgb = np.append(data_rgb, y['rgb_data'])
    print(data_rgb.shape)
    y = []

data_rgb = data_rgb.astype('float')
data_rgb = data_rgb.reshape(5000, 150, 150, 3)

print( data_rgb.shape )
print( labels.shape )

num_images, nrows, ncols, dims = data_rgb.shape